# 📱 Flutter Manga Reader - Ultimate Builder

Bu notebook orqali siz **Flutter Manga App** uchun APK faylni eng oson va ishonchli usulda tayyorlay olasiz.

**Repository:** https://github.com/UzbekApis/flutter-manga-app

---

## 🚀 Qanday ishlatiladi?

1. Yuqoridagi menyudan **Runtime** -> **Run all** ni tanlang (yoki `Ctrl+F9`).
2. Taxminan **20 daqiqa** kuting.
3. APK fayl avtomatik ravishda yuklab olinadi.

---

## 🛠️ 1. Muhitni Sozlash (Environment Setup)

Bu qadamda barcha kerakli dasturlar o'rnatiladi:
- System packages (Java, git, curl, unzip, cmake, ninja)
- Flutter SDK (Stable)
- Android SDK & Tools

In [ ]:
import os
import sys

def run_step(step_name, command, check_path=None):
    print(f"\n🚀 {step_name}...")
    result = os.system(command)
    if result != 0:
        print(f"❌ {step_name} xatolik bilan tugadi!")
        # Xatolik bo'lsa ham davom etishga harakat qilamiz, ba'zan apt-get xato berishi mumkin lekin ishlashi mumkin
    if check_path and not os.path.exists(check_path):
        print(f"❌ {step_name}: {check_path} topilmadi!")
        # Critical path yo'q bo'lsa to'xtatamiz
        if "flutter" in check_path or "android" in check_path:
             raise Exception(f"Critical requirement missing: {check_path}")
    print(f"✅ {step_name} muvaffaqiyatli!")

print("📦 Tizimni tayyorlash boshlandi...")

# 1. System packages
run_step(
    "Tizim kutubxonalarini o'rnatish",
    "apt-get update -qq && apt-get install -y -qq curl git unzip xz-utils zip libglu1-mesa openjdk-11-jdk cmake ninja-build"
)

# 2. Flutter
if not os.path.exists('/content/flutter'):
    run_step(
        "Flutter yuklash",
        "git clone https://github.com/flutter/flutter.git -b stable --depth 1",
        "/content/flutter/bin/flutter"
    )

# PATH sozlash
os.environ['PATH'] = f"{os.environ['PATH']}:/content/flutter/bin"

# 3. Android SDK
if not os.path.exists('/content/android-sdk'):
    run_step(
        "Android SDK yuklash",
        "wget -q https://dl.google.com/android/repository/commandlinetools-linux-9477386_latest.zip"
    )
    run_step(
        "Android SDK arxivdan chiqarish",
        "mkdir -p /content/android-sdk/cmdline-tools && unzip -q commandlinetools-linux-9477386_latest.zip -d /content/android-sdk/cmdline-tools && mv /content/android-sdk/cmdline-tools/cmdline-tools /content/android-sdk/cmdline-tools/latest",
        "/content/android-sdk/cmdline-tools/latest/bin/sdkmanager"
    )
    
    # Environment variables
    os.environ['ANDROID_HOME'] = '/content/android-sdk'
    os.environ['ANDROID_SDK_ROOT'] = '/content/android-sdk'
    
    run_step(
        "Android SDK paketlarini o'rnatish",
        "yes | /content/android-sdk/cmdline-tools/latest/bin/sdkmanager --sdk_root=/content/android-sdk 'platform-tools' 'platforms;android-33' 'build-tools;33.0.2'",
        "/content/android-sdk/platform-tools/adb"
    )
    
    run_step(
        "Litsenziyalarni qabul qilish",
        "yes | /content/android-sdk/cmdline-tools/latest/bin/sdkmanager --sdk_root=/content/android-sdk --licenses"
    )

print("\n✨ Muhit to'liq tayyor!")

## 📂 2. Loyihani Yuklash va Tayyorlash

GitHub'dan loyihani olamiz va kerakli kutubxonalarni o'rnatamiz.

In [ ]:
import os

# 4. Loyihani yuklash
if not os.path.exists('/content/flutter-manga-app'):
    print("📂 Loyiha yuklanmoqda...")
    !git clone https://github.com/UzbekApis/flutter-manga-app.git
else:
    print("📂 Loyiha allaqachon mavjud, yangilanmoqda...")
    os.chdir('/content/flutter-manga-app')
    !git pull

# 5. Dependencies
print("📦 Dependencies o'rnatilmoqda...")
os.chdir('/content/flutter-manga-app')

# Environment variables qayta set qilish (har ehtimolga qarshi)
os.environ['PATH'] = f"{os.environ['PATH']}:/content/flutter/bin"
os.environ['ANDROID_HOME'] = '/content/android-sdk'
os.environ['ANDROID_SDK_ROOT'] = '/content/android-sdk'

!flutter pub get

print("✅ Loyiha tayyor!")

## 🔨 3. APK Build Qilish

Eng muhim qism. Bu jarayon 5-10 daqiqa vaqt olishi mumkin.

In [ ]:
import os

print("🔨 APK Build jarayoni boshlandi...")
print("⏳ Iltimos kuting, bu biroz vaqt oladi...")

os.chdir('/content/flutter-manga-app')

# Build
!flutter build apk --release

# Check result
apk_path = '/content/flutter-manga-app/build/app/outputs/flutter-apk/app-release.apk'

if os.path.exists(apk_path):
    size_mb = os.path.getsize(apk_path) / 1024 / 1024
    print(f"\n" + "="*60)
    print("✅ BUILD MUVAFFAQIYATLI!")
    print("="*60)
    print(f"📱 APK hajmi: {size_mb:.2f} MB")
else:
    print("❌ Build xatolik bilan tugadi. Yuqoridagi loglarni tekshiring.")

## ⬇️ 4. Yuklab Olish

Agar build muvaffaqiyatli bo'lsa, APK fayl avtomatik yuklab olinadi.

In [ ]:
from google.colab import files
import os

apk_path = '/content/flutter-manga-app/build/app/outputs/flutter-apk/app-release.apk'

if os.path.exists(apk_path):
    print("⬇️ APK yuklab olinmoqda...")
    files.download(apk_path)
else:
    print("❌ Yuklab olish uchun APK fayl topilmadi.")